In [2]:
import csv
import json
import os
import random

If you're using a JSON file, you can point to it in the code block below. Otherwise, use the second code block to fill out a dict that describes the dataset you're generating. Your data should look something like this:
```
data_descriptor_json = {
  "defaultLabelColumn": "recommendation", #the default label to be predicted
  "fields": [
    {
      "id": "age", #whatever the field contains
      "type": "categorical", #or continuous, or binary
      "values": {
        #if categorical or binary
        "categories": [],
        #if continuous:
        "min": 0,
        "max": 100
      }
    }
   ]
}
```

In [ ]:
data_json_file = open('your_file.json')
data_descriptor_json = json.read(data_json_file)

In [39]:
data_descriptor_json = {
  "defaultLabelColumn": "decision",
  "fields": [
    {
      "id": "age",
      "type": "categorical",
      "values": {
        "categories": ["Under 18", "18-34", "35-60", "Over 60"]
      }
    },
    {
      "id": "gender",
      "type": "categorical",
      "values": {
        "categories": ["male", "female"]
      }
    },
    {
      "id": "race",
      "type": "categorical",
      "values": {
        "categories": ["White", "Black or African American", "Asian", "American Indian or Alaska Native", "Native Hawaiian or Pacific Islander"]
      }
    },
    {
      "id": "bleeding",
      "type": "categorical",
      "values": {
        "categories": ["yes", "no"]
      }
    },
    {
      "id": "shortness_of_breath",
      "type": "categorical",
      "values": {
        "categories": ["yes", "no"]
      }
    },
    {
      "id": "weight",
      "type": "categorical",
      "values": {
        "categories": ["under 100 lbs", "100-150 lbs", "150-200 lbs", "200-300 lbs", "Over 300 lbs"]
      }
    },
    {
      "id": "area_of_pain",
      "type": "categorical",
      "values": {
        "categories": ["head", "chest", "limb", "stomach"]
      }
    },
    {
      "id": "decision",
      "type": "categorical",
      "values": {
        "categories": ["priority", "normal", "return later"]
      }
    }
  ]
}

fields = data_descriptor_json['fields']
label = data_descriptor_json['defaultLabelColumn']




Here we set up our constants. Add more correlation values as needed - the idea is that we assign a dict to a category of the outcome we're predicting. If a certain other feature has a category that correlates to that outcome, add that feature as a key to this dict, and then give it a "values" attribute containing a list with the index of the category or categories that correspond. So, if we have a data description like this, where we're trying to determine if a patient is sick:
``` 
{ 
 "defaultLabelColumn": "is_sick",
 "fields": [
     {
         "id": "nose_status"
         "type: "categorical"
         "values": {
         "categories": [
         "runny", "bleeding", "dry", "exploded"
         ]
     },
     {
         "id": "is_sick"
         "type: "categorical"
         "values": {
         "categories": [
         "no", "maybe", "definitely", "extremely"
         ]
     },
 }
```
 And, as a rule, if a patient's nose is dry they're always fine but otherwise the worse symptoms they have the more likely they are to be sick, we can structure our correlation as follows:
 
```
correlation_values = {
    "label": data_descriptor_json['defaultLabelColumn'],
    "values": {
    #here we use the categories of outcome as keys:
        "no": {
            "correlation_per_category": { 1: 1 }, #here we have a one-to-one correlation
            "correlated_field_name": "nose_status",
            "correlated_category_values": "dry" # so if nose_status = dry, is_sick = "no
        }
     }
```

This block prints out a quick template for converting our values in order to set up our correlation relationships

In [47]:
for i in fields:
    if i['id'] == label:
        print("template_values = {'label': '%s', 'values': {" % label)
        for k in i['values']['categories']:
            print('"%s": {"correlation_per_category": {0: 0}, "correlated_fields": {"name": "field_name", "correlated_category_values": []}}' % k)
        print("}}")
    else:
        print('field: ', i['id'])
        print('categories: ', i['values']['categories'])


field:  age
categories:  ['Under 18', '18-34', '35-60', 'Over 60']
field:  gender
categories:  ['male', 'female']
field:  race
categories:  ['White', 'Black or African American', 'Asian', 'American Indian or Alaska Native', 'Native Hawaiian or Pacific Islander']
field:  bleeding
categories:  ['yes', 'no']
field:  shortness_of_breath
categories:  ['yes', 'no']
field:  weight
categories:  ['under 100 lbs', '100-150 lbs', '150-200 lbs', '200-300 lbs', 'Over 300 lbs']
field:  area_of_pain
categories:  ['head', 'chest', 'limb', 'stomach']
template_vaues = {'label': 'decision', 'values': {
"priority": {"correlation_per_category": {0: 0}, "correlated_fields": {"name": "field_name", "correlated_category_values": []}}
"normal": {"correlation_per_category": {0: 0}, "correlated_fields": {"name": "field_name", "correlated_category_values": []}}
"return later": {"correlation_per_category": {0: 0}, "correlated_fields": {"name": "field_name", "correlated_category_values": []}}
}}


In [73]:

correlation_values = {
    "label": label, 
    "values": {
        "priority": {
            "correlation_per_category": {1: .5, 2: .6, 3: .65, 4: .9},
            "correlated_fields": [
            {"name": "bleeding", "correlated_category_values": ["yes"]},
            {"name": "shortness_of_breath", "correlated_category_values": ["yes"]},
            {"name": "area_of_pain", "correlated_category_values": ["head", "chest"]},
            {"name": "weight", "correlated_category_values": ['under 100 lbs', 'Over 300 lbs']}
        ]}, 
        #just testing to make sure that different values can have the same field
        "normal": {
            "correlation_per_category": {1:1},
            "correlated_fields": [
                {"name": "weight", "correlated_category_values": ['100-150 lbs']}
            ]
        },
        "return_later": 
        {
            "correlation_per_category": {1: 1},
            "correlated_fields": 
                [{"name": "race", "correlated_category_values": ['American Indian or Alaska Native', 'Native Hawaiian or Pacific Islander'], "limit": 0.05
}],
        }
        }
    }

num_rows = 300 # the number of rows in our output dataset
outfile_name = 'medical_priority.csv' # the name of the file we're writing to, should end in '.csv'
print(json.dumps(correlation_values, indent=2))

{
  "label": "decision",
  "values": {
    "priority": {
      "correlation_per_category": {
        "1": 0.5,
        "2": 0.6,
        "3": 0.65,
        "4": 0.9
      },
      "correlated_fields": [
        {
          "name": "bleeding",
          "correlated_category_values": [
            "yes"
          ]
        },
        {
          "name": "shortness_of_breath",
          "correlated_category_values": [
            "yes"
          ]
        },
        {
          "name": "area_of_pain",
          "correlated_category_values": [
            "head",
            "chest"
          ]
        },
        {
          "name": "weight",
          "correlated_category_values": [
            "under 100 lbs",
            "Over 300 lbs"
          ]
        }
      ]
    },
    "normal": {
      "correlation_per_category": {
        "1": 1
      },
      "correlated_fields": [
        {
          "name": "weight",
          "correlated_category_values": [
            "100-150 lbs"
       

In [72]:
#write the file
with open(outfile_name, 'w') as outfile:
    writer = csv.writer(outfile, delimiter=',')
    headers = [field['id'] for field in data_descriptor_json['fields']]
    writer.writerow(headers)
    label_val_idx = headers.index(label)
    
    #TODO negative correlation? wait i guess this does work that way since we're just doing positive correlation to a different category
    
    #FIXME uhhh this is real confusing and brute forced
    #abandon hope all ye who enter here
    #definitely need to improve variable names there are about five different 'values' in there
    #reduce nesting?
    #break into functions so we can just skip if field isn't in correlation dict?
    for i in range(num_rows):
        row = []
        #object to see how highly they're correlated with diff outcomes
        correlation_counter = {}
        for field in data_descriptor_json['fields']:
            field_name = field['id']
            if field_name == label:
                row.append('')
                continue
            values = field["values"]
            categories = values["categories"]
            correlation_value_list = correlation_values["values"]

            random_num = random.random()
            random_choice = random.choice(categories)

            #check against correlation values and increment our relationship accordingly
            for key, value in correlation_value_list.items():
                    if key == 'correlation_per_category':
                        continue
                    for cfield in value['correlated_fields']:
                        if 'limit' in cfield  and field_name == cfield['name']:
                        #if there's a limit on the number of total datapoints with this value, meet it       
                            if random_num > cfield['limit']:
                                def reset_random_choice(choice):
                                    if choice in cfield['correlated_category_values']:
                                        choice = random.choice(categories)
                                        return reset_random_choice(choice)
                                    else:
                                        return choice
                                random_choice = reset_random_choice(random_choice)
                            else:
                                random_choice = random.choice(cfield['correlated_category_values'])
                            
                    for i in value['correlated_fields']:
                        for j in i['correlated_category_values']:
                            if field_name == i['name'] and random_choice == j:
                                if key in correlation_counter:
                                    correlation_counter[key] += 1
                                    break
                                else:
                                    correlation_counter[key] = 1
                                    break
                                
                        #add randomly selected thing to row if there's not already a correlation forming
            if random_choice not in correlation_counter or random_num > correlation_counter[random_choice]:
                row.append(random_choice)
            else:
                #if there is a correlation forming, select from correlated values
                random_choice = random.choice(correlation_value_list[field['id']]['values'])
                row.append(field['categories'][random_choice])
        # print('correlation counter: ', correlation_counter)
        get_decision = random.random()
        decision_value = ''
        decision_corr = 0
        #check if correlation happened, if so make more likely that we meet that condition
        print('correlation counter: ', correlation_counter)
        for key, value in correlation_value_list.items():
            if key not in correlation_counter:
                continue
            if value["correlation_per_category"][correlation_counter[key]] > decision_corr:
                decision_corr = value["correlation_per_category"][correlation_counter[key]]
                decision_value = key
        if get_decision < decision_corr:
            row[label_val_idx] = decision_value
        else:
            row[label_val_idx] = random.choice(data_descriptor_json['fields'][label_val_idx]['values']['categories'])
        print('row:', row)      
                            
                        
        writer.writerow(row)

correlation counter:  {'priority': 1}
row: ['18-34', 'male', 'White', 'no', 'no', 'under 100 lbs', 'limb', 'priority']
correlation counter:  {'return_later': 1, 'priority': 2, 'normal': 1}
row: ['18-34', 'male', 'Native Hawaiian or Pacific Islander', 'yes', 'yes', '100-150 lbs', 'stomach', 'normal']
correlation counter:  {'priority': 2}
row: ['Under 18', 'female', 'White', 'yes', 'no', '200-300 lbs', 'chest', 'priority']
correlation counter:  {'priority': 1}
row: ['35-60', 'male', 'Asian', 'no', 'no', '150-200 lbs', 'head', 'return later']
correlation counter:  {}
row: ['Under 18', 'male', 'Asian', 'no', 'no', '150-200 lbs', 'limb', 'return later']
correlation counter:  {'priority': 2, 'normal': 1}
row: ['18-34', 'male', 'Asian', 'no', 'yes', '100-150 lbs', 'chest', 'normal']
correlation counter:  {'priority': 3}
row: ['Under 18', 'male', 'White', 'no', 'yes', 'under 100 lbs', 'chest', 'priority']
correlation counter:  {}
row: ['Over 60', 'male', 'Black or African American', 'no', 'no'